In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

In [59]:
class Config:
    """Configuración centralizada del proyecto"""
    CSV_IN = Path('../db/export.csv')
    CSV_OUT = Path('../data/dataset_enriquecido.csv')
    GRAFICOS_DIR = Path('../docs/graficos/')

    COLUMN_MAP = {
        'incapermaparciar_at': 'inc_at',
        'incapermaparciar_el': 'inc_el',
        'muertes_repor_at': 'muertes',
        'nuevapensioinva_r_at': 'pen_at',
        'nuevapensioinva_r_el': 'pen_el',
        'presuaccidetrasuce': 'presuntos',
        'rela_dep': 'dep',
        'rela_indep': 'indep',
        'a_o_de_informe': 'anio',
        'mes_de_informe': 'mes'
    }
    
    MESES_MAP = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)


In [60]:
def cargar_datos(ruta):
    """Carga el dataset desde CSV"""
    try:
        df = pd.read_csv(ruta, dtype=str)
        print(f"Dataset cargado: {len(df)} registros")
        return df
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en {ruta}")
        return pd.DataFrame()

def limpiar_nombres_columnas(df):
    """Limpia y normaliza nombres de columnas"""
    df.columns = (
        df.columns.str.lower()
        .str.strip()
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace(r'[^a-z0-9]+', '_', regex=True)
        .str.strip('_')
    )
    print(f"Columnas normalizadas: {list(df.columns)}")
    return df

def renombrar_columnas(df, mapeo):
    """Renombra columnas según mapeo definido"""
    cols_existentes = {k: v for k, v in mapeo.items() if k in df.columns}
    df.rename(columns=cols_existentes, inplace=True)
    print(f"Columnas renombradas: {list(cols_existentes.values())}")
    return df

def eliminar_duplicados(df):
    """Elimina registros duplicados"""
    inicial = len(df)
    df.drop_duplicates(inplace=True)
    eliminados = inicial - len(df)
    print(f"Duplicados eliminados: {eliminados}")
    return df

def manejar_nulos(df, cols_clave):
    """Maneja valores nulos en columnas clave"""
    inicial = len(df)
    df.dropna(subset=cols_clave, inplace=True)
    eliminados = inicial - len(df)
    print(f"Registros con nulos eliminados: {eliminados}")
    return df

def convertir_numericas(df, cols):
    """Convierte columnas a tipo numérico"""
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    print(f"Columnas convertidas a numéricas: {cols}")
    return df


In [61]:
def crear_columna_fecha(df, meses_map):
    """Crea columna de fecha a partir de año y mes"""
    try:
        df['mes_num'] = df['mes'].astype(str).str.lower().map(meses_map)
        df['mes_num'] = df['mes_num'].fillna(df['mes'].astype(str).str.extract(r'(\d+)')[0].astype(float))
        df['mes_num'] = df['mes_num'].fillna(1).astype(int)
        
        df['fecha'] = pd.to_datetime(
            df['anio'].astype(str) + '-' + 
            df['mes_num'].astype(str) + '-15',
            errors='coerce'
        )
        
        df.dropna(subset=['fecha'], inplace=True)
        print(f"Columna 'fecha' creada: {len(df)} registros válidos")
        
    except Exception as e:
        print(f"Error al crear fecha: {e}")
        df['fecha'] = pd.to_datetime('2024-01-15')
    
    return df

def derivar_columnas_temporales(df):
    """Deriva columnas adicionales desde fecha"""
    df['anio'] = df['fecha'].dt.year
    df['mes_num'] = df['fecha'].dt.month
    df['dia'] = df['fecha'].dt.day
    df['trimestre'] = df['fecha'].dt.quarter
    print(f"Columnas temporales derivadas")
    return df


In [62]:
def configurar_directorio_graficos(directorio):
    """Crea directorio de gráficos si no existe"""
    directorio.mkdir(parents=True, exist_ok=True)
    print(f"Directorio de gráficos: {directorio}")

def grafico_top_actividades(df, graficos_dir):
    """Gráfico: Top 10 actividades con mayor incidentalidad"""
    if 'activec' not in df.columns or 'inc_at' not in df.columns:
        print("Columnas necesarias no encontradas para gráfico de actividades")
        return
    
    df_act = df.groupby('activec')['inc_at'].sum().nlargest(10).reset_index()
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='inc_at', y='activec', data=df_act, palette='viridis')
    plt.title('Top 10 Actividades Económicas con Mayor Incidentalidad', fontsize=14, fontweight='bold')
    plt.xlabel('Total de Incidentes de Trabajo')
    plt.ylabel('Actividad Económica')
    plt.tight_layout()
    plt.savefig(graficos_dir / '01_incidentes_por_actividad.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Gráfico 1: Incidentes por actividad")

def grafico_distribucion_incidentes(df, graficos_dir):
    """Gráfico: Distribución de incidentes"""
    if 'inc_at' not in df.columns:
        print("Columna 'inc_at' no encontrada")
        return
    
    plt.figure(figsize=(10, 6))
    sns.histplot(df['inc_at'], bins=50, kde=True, color='skyblue')
    plt.title('Distribución de Incidentes de Trabajo', fontsize=14, fontweight='bold')
    plt.xlabel('Número de Incidentes')
    plt.ylabel('Frecuencia')
    plt.yscale('log')
    plt.tight_layout()
    plt.savefig(graficos_dir / '02_histograma_inc_at.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Gráfico 2: Distribución de incidentes")

def grafico_muertes_vs_cobertura(df, graficos_dir):
    """Gráfico: Relación muertes vs cobertura"""
    if 'dep' not in df.columns or 'muertes' not in df.columns:
        print("Columnas necesarias no encontradas para gráfico muertes vs cobertura")
        return
    
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='dep', y='muertes', data=df, alpha=0.5, color='red', s=30)
    plt.title('Relación: Trabajadores Dependientes vs Muertes', fontsize=14, fontweight='bold')
    plt.xlabel('Trabajadores Dependientes')
    plt.ylabel('Número de Muertes')
    plt.tight_layout()
    plt.savefig(graficos_dir / '03_muertes_vs_cobertura.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Gráfico 3: Muertes vs cobertura")

def grafico_tendencia_anual(df, graficos_dir):
    """Gráfico: Tendencia temporal anual"""
    if 'anio' not in df.columns or 'inc_at' not in df.columns or 'muertes' not in df.columns:
        print("Columnas necesarias no encontradas para gráfico de tendencia")
        return
    
    df_anual = df.groupby('anio')[['inc_at', 'muertes']].sum().reset_index()
    
    plt.figure(figsize=(10, 6))
    plt.plot(df_anual['anio'], df_anual['inc_at'], marker='o', label='Incidentes', linewidth=2)
    plt.plot(df_anual['anio'], df_anual['muertes'], marker='s', label='Muertes', color='red', linewidth=2)
    plt.title('Tendencia Anual: Incidentes y Muertes', fontsize=14, fontweight='bold')
    plt.xlabel('Año')
    plt.ylabel('Total')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(graficos_dir / '04_tendencia_anual.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Gráfico 4: Tendencia anual")

def generar_estadisticas(df, cols):
    """Genera estadísticas descriptivas"""
    cols_existentes = [c for c in cols if c in df.columns]
    if cols_existentes:
        print("\n" + "="*70)
        print("ESTADÍSTICAS DESCRIPTIVAS")
        print("="*70)
        print(df[cols_existentes].describe().transpose())
    else:
        print("No se encontraron columnas para estadísticas")


In [63]:
def pipeline_completo():
    """Ejecuta el pipeline completo de procesamiento"""
    print("\n" + "="*70)
    print("INICIANDO PIPELINE DE PROCESAMIENTO")
    print("="*70 + "\n")
    
    # 1. CARGA
    print("[ 1/4 ] CARGA DE DATOS")
    df = cargar_datos(Config.CSV_IN)
    if df.empty:
        return
    
    # 2. LIMPIEZA
    print("\n[ 2/4 ] LIMPIEZA DE DATOS")
    df = limpiar_nombres_columnas(df)
    df = renombrar_columnas(df, Config.COLUMN_MAP)
    df = eliminar_duplicados(df)
    df = manejar_nulos(df, ['activec', 'dpto'])
    
    cols_numericas = ['inc_at', 'inc_el', 'muertes', 'pen_at', 'pen_el', 'presuntos', 'dep', 'indep']
    df = convertir_numericas(df, cols_numericas)
    
    # 3. ENRIQUECIMIENTO
    print("\n[ 3/4 ] ENRIQUECIMIENTO DE DATOS")
    df = crear_columna_fecha(df, Config.MESES_MAP)
    df = derivar_columnas_temporales(df)
    
    # Guardar dataset procesado
    Config.CSV_OUT.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(Config.CSV_OUT, index=False, encoding='utf-8')
    print(f"Dataset enriquecido guardado: {Config.CSV_OUT}")
    
    # 4. VISUALIZACIÓN Y EDA
    print("\n[ 4/4 ] ANÁLISIS EXPLORATORIO Y VISUALIZACIÓN")
    configurar_directorio_graficos(Config.GRAFICOS_DIR)
    
    generar_estadisticas(df, cols_numericas)
    
    grafico_top_actividades(df, Config.GRAFICOS_DIR)
    grafico_distribucion_incidentes(df, Config.GRAFICOS_DIR)
    grafico_muertes_vs_cobertura(df, Config.GRAFICOS_DIR)
    grafico_tendencia_anual(df, Config.GRAFICOS_DIR)
    
    print("\n" + "="*70)
    print("PIPELINE COMPLETADO EXITOSAMENTE")
    print("="*70)
    print(f"\nRegistros finales: {len(df)}")
    print(f"Datos: {Config.CSV_OUT}")
    print(f"Gráficos: {Config.GRAFICOS_DIR}")


In [64]:
if __name__ == "__main__":
    pipeline_completo()


INICIANDO PIPELINE DE PROCESAMIENTO

[ 1/4 ] CARGA DE DATOS
Dataset cargado: 88620 registros

[ 2/4 ] LIMPIEZA DE DATOS
Columnas normalizadas: ['dpto', 'mpio', 'codigo_de_la_arl', 'ao_de_informe', 'mes_de_informe', 'activec', 'rela_dep', 'rela_indep', 'presuaccidetrasuce', 'muertes_repor_at', 'nuevapensioinva_r_at', 'nuevapensioinva_r_el', 'incapermaparciar_at', 'incapermaparciar_el']
Columnas renombradas: ['inc_at', 'inc_el', 'muertes', 'pen_at', 'pen_el', 'presuntos', 'dep', 'indep', 'mes']
Duplicados eliminados: 0
Registros con nulos eliminados: 0
Columnas convertidas a numéricas: ['inc_at', 'inc_el', 'muertes', 'pen_at', 'pen_el', 'presuntos', 'dep', 'indep']

[ 3/4 ] ENRIQUECIMIENTO DE DATOS
Error al crear fecha: 'anio'
Columnas temporales derivadas
Dataset enriquecido guardado: ..\data\dataset_enriquecido.csv

[ 4/4 ] ANÁLISIS EXPLORATORIO Y VISUALIZACIÓN
Directorio de gráficos: ..\docs\graficos

ESTADÍSTICAS DESCRIPTIVAS
             count       mean         std  min  25%  50% 

C:\Users\aplazas\AppData\Local\Temp\ipykernel_25088\2435080152.py:15: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='inc_at', y='activec', data=df_act, palette='viridis')


Gráfico 1: Incidentes por actividad
Gráfico 2: Distribución de incidentes
Gráfico 3: Muertes vs cobertura
Gráfico 4: Tendencia anual

PIPELINE COMPLETADO EXITOSAMENTE

Registros finales: 88620
Datos: ..\data\dataset_enriquecido.csv
Gráficos: ..\docs\graficos
